In [1]:
import time

from sklearn.datasets import load_iris, fetch_20newsgroups, fetch_california_housing
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.feature_extraction import DictVectorizer
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score

使用load可以直接加载的内存的，数据集比较小，并不会保存到本地磁盘

使用fetch适用于比较大的数据集，下载下来后会存在本地磁盘，下一次就不会再连接sklearn的服务器

In [2]:
#鸢尾花数据集，查看特征，目标，样本量

li = load_iris()

print("获取特征值")
print(type(li.data))
print('-' * 50)
print(li.data.shape) # 150个样本，4个特征,一般看shape
li.data

获取特征值
<class 'numpy.ndarray'>
--------------------------------------------------
(150, 4)


array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [3]:
print("目标值")
#数据集的标签数组，通常是一个一维数组，包含每个样本对应的类别编号（如 0、1、2）。对于鸢尾花数据集，它包含 150 个整数值，代表三种鸢尾花
print(li.target)
print('-' * 50)
#数据集的详细描述（字符串），包含数据集来源、特征说明、类别含义等信息
print(li.DESCR)
print('-' * 50)
#特征名称列表，例如 ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']，用于说明每个特征的含义。
print(li.feature_names)  # 重点,特征名字
print('-' * 50)
#目标类别名称列表，例如 ['setosa', 'versicolor', 'virginica']，将目标值（0、1、2）映射为具体的类别名称。
print(li.target_names) # 目标名字

目标值
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
--------------------------------------------------
.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

:Number of Instances: 150 (50 in each of three classes)
:Number of Attributes: 4 numeric, predictive attributes and the class
:Attribute Information:
    - sepal length in cm
    - sepal width in cm
    - petal length in cm
    - petal width in cm
    - class:
            - Iris-Setosa
            - Iris-Versicolour
            - Iris-Virginica

:Summary Statistics:

============== ==== ==== ======= ===== ====================
                Min  Max   Mean    SD   Class Correlation
============== ==== ==== ======= ===== ====================

In [4]:
print(li.data.shape)
li.target.shape

(150, 4)


(150,)

In [5]:
# 注意返回值, 训练集：train、x_train、y_train        测试集：test、x_test、y_test，顺序千万别搞错了
# 默认是乱序的,random_state为了确保两次的随机策略一致，就会得到相同的随机数据，往往会带上
x_train, x_test, y_train, y_test = train_test_split(li.data, li.target, test_size=0.25, random_state=1)

# print("训练集特征值和目标值：", x_train, y_train)
print("训练集特征值shape", x_train.shape)
print('-'*50)
# print("测试集特征值和目标值：", x_test, y_test)
print("测试集特征值shape", x_test.shape)

训练集特征值shape (112, 4)
--------------------------------------------------
测试集特征值shape (38, 4)


In [6]:
150*0.25

37.5

In [7]:
# 下面是比较大的数据，需要下载一会，20类新闻
#subset代表下载的数据集类型，默认是train，只有训练集
news = fetch_20newsgroups(subset='all', data_home='data')
# print(news.feature_names)  #这个数据集是没有的，因为没有特征，只有文本数据
# print(news.DESCR)
print('第一个样本')
print(news.data[0])

第一个样本
From: Mamatha Devineni Ratnam <mr47+@andrew.cmu.edu>
Subject: Pens fans reactions
Organization: Post Office, Carnegie Mellon, Pittsburgh, PA
Lines: 12
NNTP-Posting-Host: po4.andrew.cmu.edu



I am sure some bashers of Pens fans are pretty confused about the lack
of any kind of posts about the recent Pens massacre of the Devils. Actually,
I am  bit puzzled too and a bit relieved. However, I am going to put an end
to non-PIttsburghers' relief with a bit of praise for the Pens. Man, they
are killing those Devils worse than I thought. Jagr just showed you why
he is much better than his regular season stats. He is also a lot
fo fun to watch in the playoffs. Bowman should let JAgr have a lot of
fun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final
regular season game.          PENS RULE!!!




In [8]:
print('特征类型')
print(type(news.data))
print('-' * 50)
print(news.target[0:15])
from pprint import pprint
pprint(list(news.target_names))

特征类型
<class 'list'>
--------------------------------------------------
[10  3 17  3  4 12  4 10 10 19 19 11 19 13  0]
['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']


In [9]:
len(news.target_names)

20

In [10]:
print('-' * 50)
print(len(news.data))
print('新闻所有的标签')
print(news.target)
print('-' * 50)
print(min(news.target), max(news.target))

--------------------------------------------------
18846
新闻所有的标签
[10  3 17 ...  3  1  7]
--------------------------------------------------
0 19


In [11]:
#因为新版本sklearn去掉了这个数据集，因此不再讲解
# 接着来看回归的数据,是波士顿房价
# lb = load_boston()
#
# print("获取特征值")
# print(lb.data[0])  #第一个样本特征值
# print(lb.data.shape)
# print('-' * 50)
# print("目标值")
# print(lb.target)
# print('-' * 50)
# print(lb.DESCR)
# print('-' * 50)
# print(lb.feature_names)
# print('-' * 50)
# 回归问题没这个,打印这个会报错
# print(lb.target_names)

In [12]:
house=fetch_california_housing(data_home='data')
print("获取特征值")
print(house.data[0])  #第一个样本特征值
print('样本的形状')
print(house.data.shape)
print('-' * 50)


获取特征值
[   8.3252       41.            6.98412698    1.02380952  322.
    2.55555556   37.88       -122.23      ]
样本的形状
(20640, 8)
--------------------------------------------------


In [13]:
print("目标值")
print(house.target[0:10])
print('-' * 50)
print(house.DESCR)
print('-' * 50)
print(house.feature_names)
print('-' * 50)

目标值
[4.526 3.585 3.521 3.413 3.422 2.697 2.992 2.414 2.267 2.611]
--------------------------------------------------
.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

:Number of Instances: 20640

:Number of Attributes: 8 numeric, predictive attributes and the target

:Attribute Information:
    - MedInc        median income in block group
    - HouseAge      median house age in block group
    - AveRooms      average number of rooms per household
    - AveBedrms     average number of bedrooms per household
    - Population    block group population
    - AveOccup      average number of household members
    - Latitude      block group latitude
    - Longitude     block group longitude

:Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in h

# 2 分类估计器

In [14]:
np.sqrt(15*15+14*14)

np.float64(20.518284528683193)

In [15]:
# K近邻
"""
K-近邻预测用户签到位置
:return:None
"""
# 读取数据
data = pd.read_csv("./data/FBlocation/train.csv")

print(data.head(10))
print(data.shape)
print(data.info())
# 处理数据
# 1、缩小数据,查询数据,为了减少计算时间
data = data.query("x > 1.0 &  x < 1.25 & y > 2.5 & y < 2.75")

   row_id       x       y  accuracy    time    place_id
0       0  0.7941  9.0809        54  470702  8523065625
1       1  5.9567  4.7968        13  186555  1757726713
2       2  8.3078  7.0407        74  322648  1137537235
3       3  7.3665  2.5165        65  704587  6567393236
4       4  4.0961  1.1307        31  472130  7440663949
5       5  3.8099  1.9586        75  178065  6289802927
6       6  6.3336  4.3720        13  666829  9931249544
7       7  5.7409  6.7697        85  369002  5662813655
8       8  4.3114  6.9410         3  166384  8471780938
9       9  6.3414  0.0758        65  400060  1253803156
(29118021, 6)
<class 'pandas.DataFrame'>
RangeIndex: 29118021 entries, 0 to 29118020
Data columns (total 6 columns):
 #   Column    Dtype  
---  ------    -----  
 0   row_id    int64  
 1   x         float64
 2   y         float64
 3   accuracy  int64  
 4   time      int64  
 5   place_id  int64  
dtypes: float64(2), int64(4)
memory usage: 1.3 GB
None


In [16]:
data.shape

(17710, 6)

In [17]:
data.describe()

,row_id,x,y,accuracy,time,place_id
count,1.771000e+04,17710.000000,17710.000000,17710.000000,17710.000000,1.771000e+04
mean,1.450569e+07,1.122538,2.632309,82.482101,397551.263128,5.129895e+09
std,8.353805e+06,0.077086,0.070144,113.613227,234601.097883,2.357399e+09
min,6.000000e+02,1.000100,2.500100,1.000000,119.000000,1.012024e+09
25%,7.327816e+06,1.049200,2.573800,25.000000,174069.750000,3.312464e+09
50%,1.443071e+07,1.123300,2.642300,62.000000,403387.500000,5.261906e+09
75%,2.163463e+07,1.190500,2.687800,75.000000,602111.750000,6.766325e+09
max,2.911215e+07,1.249900,2.749900,1004.000000,786218.000000,9.980711e+09


In [18]:
# 处理时间的数据
time_value = pd.to_datetime(data['time'], unit='s')

print(time_value.head(10))  #最大时间是1月10号

600    1970-01-01 18:09:40
957    1970-01-10 02:11:10
4345   1970-01-05 15:08:02
4735   1970-01-06 23:03:03
5580   1970-01-09 11:26:50
6090   1970-01-02 16:25:07
6234   1970-01-04 15:52:57
6350   1970-01-01 10:13:36
7468   1970-01-09 15:26:06
8478   1970-01-08 23:52:02
Name: time, dtype: datetime64[s]


In [19]:
# 把日期格式转换成字典格式，把年，月，日，时，分，秒转换为字典格式，
time_value = pd.DatetimeIndex(time_value)

print('-' * 50)
print(time_value[0:10])

--------------------------------------------------
DatetimeIndex(['1970-01-01 18:09:40', '1970-01-10 02:11:10',
               '1970-01-05 15:08:02', '1970-01-06 23:03:03',
               '1970-01-09 11:26:50', '1970-01-02 16:25:07',
               '1970-01-04 15:52:57', '1970-01-01 10:13:36',
               '1970-01-09 15:26:06', '1970-01-08 23:52:02'],
              dtype='datetime64[s]', name='time', freq=None)


In [20]:
data.shape

(17710, 6)

In [21]:
print('-' * 50)
# 构造一些特征，执行的警告是因为我们的操作是复制，loc是直接放入
print(type(data))
# data['day'] = time_value.day
# data['hour'] = time_value.hour
# data['weekday'] = time_value.weekday
#日期，是否是周末，小时对于个人行为的影响是较大的(例如吃饭时间去饭店，看电影时间去电影院等),所以才做下面的处理
data.insert(data.shape[1], 'day', time_value.day) #data.shape[1]是代表插入到最后的意思,一个月的哪一天
data.insert(data.shape[1], 'hour', time_value.hour)#是否去一个地方打卡，早上，中午，晚上是有影响的
data.insert(data.shape[1], 'weekday', time_value.weekday) #0代表周一，6代表周日，星期几
# 把时间戳特征删除
data = data.drop(['time'], axis=1)

--------------------------------------------------
<class 'pandas.DataFrame'>


In [22]:
print('-' * 50)
data.head()

--------------------------------------------------


,row_id,x,y,accuracy,place_id,day,hour,weekday
600,600,1.2214,2.7023,17,6683426742,1,18,3
957,957,1.1832,2.6891,58,6683426742,10,2,5
4345,4345,1.1935,2.6550,11,6889790653,5,15,0
4735,4735,1.1452,2.6074,49,6822359752,6,23,1
5580,5580,1.0089,2.7287,19,1527921905,9,11,4


In [23]:
#星期天，实际 weekday 的值是6
per = pd.Period('1970-01-01 18:00', 'h') #表示一个以小时为频率的时间段，起始时间为 1970-01-01 18:00
per.weekday #是 Period 对象的一个属性，返回该时间段起始日期对应的星期几，0代表周一，6代表周日
#1970年1月1日是星期四，返回3

3

In [24]:
#观察数据，看下是否有空值，异常值
data.describe()

,row_id,x,y,accuracy,place_id,day,hour,weekday
count,1.771000e+04,17710.000000,17710.000000,17710.000000,1.771000e+04,17710.000000,17710.000000,17710.000000
mean,1.450569e+07,1.122538,2.632309,82.482101,5.129895e+09,5.101863,11.485545,3.092377
std,8.353805e+06,0.077086,0.070144,113.613227,2.357399e+09,2.709287,6.932195,1.680218
min,6.000000e+02,1.000100,2.500100,1.000000,1.012024e+09,1.000000,0.000000,0.000000
25%,7.327816e+06,1.049200,2.573800,25.000000,3.312464e+09,3.000000,6.000000,2.000000
50%,1.443071e+07,1.123300,2.642300,62.000000,5.261906e+09,5.000000,12.000000,3.000000
75%,2.163463e+07,1.190500,2.687800,75.000000,6.766325e+09,7.000000,17.000000,4.000000
max,2.911215e+07,1.249900,2.749900,1004.000000,9.980711e+09,10.000000,23.000000,6.000000


In [25]:
# # 把签到数量少于n个目标位置删除，place_id是标签，即目标值
place_count = data.groupby('place_id').count()
place_count

,row_id,x,y,accuracy,day,hour,weekday
place_id,,,,,,,
1012023972,1,1,1,1,1,1,1
1057182134,1,1,1,1,1,1,1
1059958036,3,3,3,3,3,3,3
1085266789,1,1,1,1,1,1,1
1097200869,1044,1044,1044,1044,1044,1044,1044
...,...,...,...,...,...,...,...
9904182060,1,1,1,1,1,1,1
9915093501,1,1,1,1,1,1,1
9946198589,1,1,1,1,1,1,1


In [26]:
place_count['x'].describe() #打卡地点总计805个，50%打卡小于2次

count     805.000000
mean       22.000000
std        88.955632
min         1.000000
25%         1.000000
50%         2.000000
75%         5.000000
max      1044.000000
Name: x, dtype: float64

In [27]:
# 把index变为0,1,2，3,4,5,6这种效果，从零开始排，原来的index是row_id
#只选择去的人大于3的数据，认为1,2,3的是噪音，这个地方去的人很少，不用推荐给其他人
tf = place_count[place_count.row_id > 3].reset_index()
tf  #剩余的签到地点

,place_id,row_id,x,y,accuracy,day,hour,weekday
0,1097200869,1044,1044,1044,1044,1044,1044,1044
1,1228935308,120,120,120,120,120,120,120
2,1267801529,58,58,58,58,58,58,58
3,1278040507,15,15,15,15,15,15,15
4,1285051622,21,21,21,21,21,21,21
...,...,...,...,...,...,...,...,...
234,9741307878,5,5,5,5,5,5,5
235,9753855529,21,21,21,21,21,21,21
236,9806043737,6,6,6,6,6,6,6
237,9809476069,23,23,23,23,23,23,23


In [28]:
# 根据设定的地点目标值，对原本的样本进行过滤
#isin可以过滤某一列要在一组值
data = data[data['place_id'].isin(tf.place_id)]#检查 data 中每一行的 place_id 值是否存在于 tf.place_id
data.shape

(16918, 8)

In [29]:
# # 取出数据当中的特征值和目标值
y = data['place_id']
# 删除目标值，保留特征值，
x = data.drop(['place_id'], axis=1)
# 删除无用的特征值，row_id是索引,这就是噪音
x = x.drop(['row_id'], axis=1)
print(x.shape)
print(x.columns)

(16918, 6)
Index(['x', 'y', 'accuracy', 'day', 'hour', 'weekday'], dtype='str')


# 上面预处理完成

In [30]:
# li = load_iris()
# x,y=li.data,li.target

In [31]:
# 进行数据的分割训练集合测试集
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=1)

# 特征工程（标准化）,下面3行注释，一开始我们不进行标准化，看下效果，目标值要不要标准化？
std = StandardScaler()
# #
# # # 对测试集和训练集的特征值进行标准化,服务于knn fit
x_train = std.fit_transform(x_train)
# # transform返回的是copy，不在原有的输入对象中去修改
# print(id(x_test))
print(std.mean_)
print(std.var_)
x_test = std.transform(x_test)  #transfrom不再进行均值和方差的计算，是在原有的基础上去标准化
print('-' * 50)
# print(id(x_test))
print(std.mean_)
print(std.var_)

[ 1.12295735  2.63237278 81.34938525  5.10064628 11.44293821  3.10135561]
[5.98489138e-03 4.86857391e-03 1.19597480e+04 7.32837915e+00
 4.83742660e+01 2.81838404e+00]
--------------------------------------------------
[ 1.12295735  2.63237278 81.34938525  5.10064628 11.44293821  3.10135561]
[5.98489138e-03 4.86857391e-03 1.19597480e+04 7.32837915e+00
 4.83742660e+01 2.81838404e+00]


In [32]:
x_train.shape

(12688, 6)

In [33]:
# # 进行算法流程 # 超参数，可以通过设置n_neighbors=5，来调整结果好坏
knn = KNeighborsClassifier(n_neighbors=6)

# # fit， predict,score，训练，knn的fit是不训练的，只是把训练集的特征值和目标值放入到内存中
knn.fit(x_train, y_train)
# # #
# # # 得出预测结果
y_predict = knn.predict(x_test)
# #
print("预测的目标签到位置为：", y_predict[0:10])
# # #
# # # # 得出准确率,是评估指标
print("预测的准确率:", knn.score(x_test, y_test))
# print(y_predict)
# y_test

预测的目标签到位置为： [5689129232 1097200869 2355236719 9632980559 6424972551 4022692381
 8048985799 6683426742 1435128522 3312463746]
预测的准确率: 0.484160756501182


In [34]:
print(max(time_value))

1970-01-10 02:23:38


# 调超参的方法，网格搜索

In [36]:
#网格搜索时讲解
# # 构造一些参数（超参）的值进行搜索
param = {"n_neighbors": [3, 5, 10, 12, 15],'weights':['uniform', 'distance']}
#uniform表示所有邻居权重相同，distance表示按距离加权
# 进行网格搜索，cv=3是3折交叉验证，将训练集分成3份，用其中2折训练，1折验证
gc = GridSearchCV(knn, param_grid=param, cv=3)

gc.fit(x_train, y_train)  #你给它的 x_train，它又分为训练集，验证集

# 预测准确率，为了给大家看看
print("在测试集上准确率：", gc.score(x_test, y_test))

print("在交叉验证当中最好的结果：", gc.best_score_) #最好的结果

print("选择最好的模型是：", gc.best_estimator_) #最好的模型,告诉你用了哪些参数

print("每个超参数每次交叉验证的结果：")
gc.cv_results_

D:\Software\Python312\Lib\site-packages\sklearn\model_selection\_split.py:813: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


在测试集上准确率： 0.49763593380614657
在交叉验证当中最好的结果： 0.4816362349278435
选择最好的模型是： KNeighborsClassifier(n_neighbors=12, weights='distance')
每个超参数每次交叉验证的结果：


{'mean_fit_time': array([0.0063444 , 0.00654268, 0.0066282 , 0.00688402, 0.00531864,
        0.00558376, 0.0055813 , 0.00526897, 0.00635823, 0.00602253]),
 'std_fit_time': array([0.00023355, 0.00069085, 0.00048594, 0.00030266, 0.00019012,
        0.0007681 , 0.00034555, 0.00019636, 0.00025493, 0.00072571]),
 'mean_score_time': array([0.02587605, 0.03642344, 0.03067565, 0.04339163, 0.03800193,
        0.05377587, 0.04245607, 0.05919925, 0.04847415, 0.06221867]),
 'std_score_time': array([0.00042897, 0.00050513, 0.0004416 , 0.00079259, 0.00062367,
        0.00011108, 0.00033887, 0.0008315 , 0.00090961, 0.00155459]),
 'param_n_neighbors': masked_array(data=[3, 3, 5, 5, 10, 10, 12, 12, 15, 15],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_value=999999),
 'param_weights': masked_array(data=['uniform', 'distance', 'uniform', 'distance',
                    'uniform', 'distance', 'uniform', 'distance',
            

In [37]:
"""
朴素贝叶斯进行文本分类
:return: None
"""
news = fetch_20newsgroups(subset='all', data_home='data')

print(len(news.data))  #样本数，包含的特征
print('-'*50)
print(news.data[0]) #第一个样本 特征
print('-'*50)
print(news.target) #标签
print(np.unique(news.target)) #标签的类别
print(news.target_names) #标签的名字

18846
--------------------------------------------------
From: Mamatha Devineni Ratnam <mr47+@andrew.cmu.edu>
Subject: Pens fans reactions
Organization: Post Office, Carnegie Mellon, Pittsburgh, PA
Lines: 12
NNTP-Posting-Host: po4.andrew.cmu.edu



I am sure some bashers of Pens fans are pretty confused about the lack
of any kind of posts about the recent Pens massacre of the Devils. Actually,
I am  bit puzzled too and a bit relieved. However, I am going to put an end
to non-PIttsburghers' relief with a bit of praise for the Pens. Man, they
are killing those Devils worse than I thought. Jagr just showed you why
he is much better than his regular season stats. He is also a lot
fo fun to watch in the playoffs. Bowman should let JAgr have a lot of
fun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final
regular season game.          PENS RULE!!!


----------------------------------------

In [40]:
print('-'*50)
# 进行数据分割
x_train, x_test, y_train, y_test = train_test_split(news.data, news.target, test_size=0.25, random_state=1)
#测试集占比为25％
# 对数据集进行特征抽取
tf = TfidfVectorizer()

# 以训练集当中的词的列表进行每篇文章重要性统计['a','b','c','d']
x_train = tf.fit_transform(x_train)
#针对特征内容，可以自行打印，下面的打印可以得到特征数目，总计有15万特征
print(len(tf.get_feature_names_out()))

--------------------------------------------------
153196


In [41]:
print(tf.get_feature_names_out()[100000])

murky


In [46]:
print(tf.get_feature_names_out()[99990:100000])

['murderer' 'murderers' 'murderes' 'murdering' 'murderous' 'murders'
 'murdoch' 'muriel' 'murillo' 'murkier']


In [60]:
import time
# 进行朴素贝叶斯算法的预测,alpha是拉普拉斯平滑系数，分子和分母加上一个系数，分母加alpha*特征词数目
mlt = MultinomialNB(alpha=1.0) #多项式朴素贝叶斯分类器
#alpha=1.0普拉斯平滑系数，用于解决零概率问题。在计算条件概率时，分子加 alpha，分母加 alpha * n_features（特征词数目），使概率估计更平滑，避免因某个特征在训练集中未出现而导致概率为零。

# print(x_train.toarray())
# 训练
start=time.time()
mlt.fit(x_train, y_train)
end=time.time()
end-start #统计训练时间

0.08612966537475586

In [61]:
x_transform_test = tf.transform(x_test)  #特征数目不发生改变
print(len(tf.get_feature_names_out())) #查看特征数目

153196


In [67]:
start=time.time()
y_predict = mlt.predict(x_transform_test)

print("预测的前面10篇文章类别为：", y_predict[0:10])

# 得出准确率,这个很难提高准确率，为什么呢？
print("准确率为：", mlt.score(x_transform_test, y_test))
end=time.time()
end-start

预测的前面10篇文章类别为： [16 19 18  1  9 15  1  2 16 13]
准确率为： 0.8518675721561969


0.03627514839172363

In [68]:
#预测的文章数目
len(y_predict)

4712

In [69]:
# 目前这个场景我们不需要召回率，support是真实是那个类别的有多少个样本
print(classification_report(y_test, y_predict,target_names=news.target_names))

                          precision    recall  f1-score   support

             alt.atheism       0.91      0.77      0.83       199
           comp.graphics       0.83      0.79      0.81       242
 comp.os.ms-windows.misc       0.89      0.83      0.86       263
comp.sys.ibm.pc.hardware       0.80      0.83      0.81       262
   comp.sys.mac.hardware       0.90      0.88      0.89       234
          comp.windows.x       0.92      0.85      0.88       230
            misc.forsale       0.96      0.67      0.79       257
               rec.autos       0.90      0.87      0.88       265
         rec.motorcycles       0.90      0.95      0.92       251
      rec.sport.baseball       0.89      0.96      0.93       226
        rec.sport.hockey       0.95      0.98      0.96       262
               sci.crypt       0.76      0.97      0.85       257
         sci.electronics       0.84      0.80      0.82       229
                 sci.med       0.97      0.86      0.91       249
         

In [71]:
y_test.shape #测试集中有多少样本

(4712,)

In [73]:
y_test1 = np.where(y_test == 0, 1, 0)
#np.where(condition, x, y) 是一个条件函数，如果condition为True，返回x，否则返回y
print(y_test1.sum()) #label为0的样本数

199


In [76]:
y_predict1 = np.where(y_predict == 0, 1, 0)
print(y_predict1.sum())

168


In [79]:
(y_test1*y_predict1).sum() #只有同时满足真实为 0 且预测为 0 的位置结果才为 1，否则为 0

np.int64(153)

In [78]:
153/168

0.9107142857142857

In [80]:
153/199

0.7688442211055276

In [81]:
max(y_test),min(y_test)

(np.int64(19), np.int64(0))

In [82]:
# 把0-19总计20个分类，变为0和1
# 5是可以改为0到19的
y_test1 = np.where(y_test == 5, 1, 0)
print(y_test1.sum()) #label为5的样本数
y_predict1 = np.where(y_predict == 5, 1, 0)
print(y_predict1.sum())
# roc_auc_score的y_test只能是二分类,针对多分类如何计算AUC
print("AUC指标：", roc_auc_score(y_test1, y_predict1))

230
214
AUC指标： 0.924078924393225


In [83]:
y_test1,y_predict1

(array([0, 0, 0, ..., 0, 0, 0], shape=(4712,)),
 array([0, 0, 0, ..., 0, 0, 0], shape=(4712,)))

In [87]:
#算多分类的精确率，召回率，F1-score
FP=np.where((np.array(y_test1)-np.array(y_predict1))==-1,1,0).sum()   #FP是18
TP=y_predict1.sum()-FP #TP是196
print(TP)
FN=np.where((np.array(y_test1)-np.array(y_predict1))==1,1,0).sum() #FN是34
print(FN)#FN是1
TN=np.where(y_test1==0,1,0).sum()-FP  #4464
print(TN)

196
34
4464


In [88]:
TP/(TP+FP) #精确率

np.float64(0.9158878504672897)

In [89]:
TP/(TP+FN)  #召回率

np.float64(0.8521739130434782)

In [90]:
#F1-score
2*TP/(2*TP+FP+FN)

np.float64(0.8828828828828829)

In [91]:
del news
del x_train
del x_test
del y_test
del y_predict
del tf

# 3 决策树

In [92]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction import DictVectorizer
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np

In [93]:
np.log2(1/32)

np.float64(-5.0)

In [94]:
1 / 2 * np.log2(1 /2) + 1 / 2 * np.log2(1 /2)

np.float64(-1.0)

In [95]:
1 / 3 * np.log2(1 / 3) + 2 / 3 * np.log2(2 / 3)

np.float64(-0.9182958340544896)

In [96]:
0.01 * np.log2(0.01) + 0.99 * np.log2(0.99)

np.float64(-0.08079313589591118)

In [98]:
"""
决策树对泰坦尼克号进行生死预测
:return: None
"""
# 获取数据
titan = pd.read_csv("./data/titanic.txt")
titan.info()

<class 'pandas.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   row.names  1313 non-null   int64  
 1   pclass     1313 non-null   str    
 2   survived   1313 non-null   int64  
 3   name       1313 non-null   str    
 4   age        633 non-null    float64
 5   embarked   821 non-null    str    
 6   home.dest  754 non-null    str    
 7   room       77 non-null     str    
 8   ticket     69 non-null     str    
 9   boat       347 non-null    str    
 10  sex        1313 non-null   str    
dtypes: float64(1), int64(2), str(8)
memory usage: 113.0 KB


In [101]:
# 处理数据，找出特征值和目标值
x = titan[['pclass', 'age', 'sex']]

y = titan['survived']
print(x.info())  # 用来判断是否有空值
print('-'*50)
print(x.describe(include='all'))

<class 'pandas.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   pclass  1313 non-null   str    
 1   age     633 non-null    float64
 2   sex     1313 non-null   str    
dtypes: float64(1), str(2)
memory usage: 30.9 KB
None
--------------------------------------------------
       pclass         age   sex
count    1313  633.000000  1313
unique      3         NaN     2
top       3rd         NaN  male
freq      711         NaN   850
mean      NaN   31.194181   NaN
std       NaN   14.747525   NaN
min       NaN    0.166700   NaN
25%       NaN   21.000000   NaN
50%       NaN   30.000000   NaN
75%       NaN   41.000000   NaN
max       NaN   71.000000   NaN


In [102]:
x.loc[:,'age'].max()

np.float64(71.0)

In [103]:
# 一定要进行缺失值处理,填为均值
mean=x['age'].mean()
x.loc[:,'age']=x.loc[:,'age'].fillna(mean)

In [104]:
x.info()

<class 'pandas.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   pclass  1313 non-null   str    
 1   age     1313 non-null   float64
 2   sex     1313 non-null   str    
dtypes: float64(1), str(2)
memory usage: 30.9 KB


In [106]:
# 分割数据集到训练集和测试集
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=4)
print(x_train.head())

    pclass        age     sex
598    2nd  30.000000    male
246    1st  62.000000    male
905    3rd  31.194181  female
300    1st  31.194181  female
509    2nd  64.000000    male


In [107]:
type(x_train)

pandas.DataFrame

In [108]:
sum(y_train)

334

In [109]:
#性别是女性的数量
x_train[x_train['sex'] == 'female'].count()

pclass    341
age       341
sex       341
dtype: int64

In [110]:
y_train

598     0
246     0
905     0
300     0
509     0
       ..
360     0
709     0
439     0
174     0
1146    0
Name: survived, Length: 984, dtype: int64

In [112]:
#女性中存活的情况对比
z=x_train.copy() #z是为了把特征和目标存储到一起，创建了一个独立的副本，避免后续操作影响原始数据。
z['survived'] = y_train #把目标值存储到z中，这一行将目标值作为新列 'survived' 添加到 z 中，使特征和目标放在同一个 DataFrame 里，便于分组统计。
z[z['sex'] == 'female']['survived'].value_counts() #男性中存活的情况
#筛选出性别为女性的所有行。对筛选后的结果取 'survived' 列，然后调用 .value_counts() 统计该列中每个值的出现次数。

survived
1    230
0    111
Name: count, dtype: int64

In [113]:
y_train.value_counts() #没存活的是650，存活的是334

survived
0    650
1    334
Name: count, dtype: int64

In [114]:
x_train.loc[:,'sex'].value_counts()

sex
male      643
female    341
Name: count, dtype: int64

In [115]:
230/(230+111)

0.6744868035190615

In [118]:
x_train

,pclass,age,sex
598,2nd,30.000000,male
246,1st,62.000000,male
905,3rd,31.194181,female
300,1st,31.194181,female
509,2nd,64.000000,male
...,...,...,...
360,2nd,31.194181,male
709,3rd,28.000000,male
439,2nd,34.000000,male
174,1st,46.000000,male


In [120]:
x_train.to_dict(orient="records") #把df变为字典，样本变为一个一个的字典，字典中列名变为键
#"records" 表示生成一个由行字典组成的列表

[{'pclass': '2nd', 'age': 30.0, 'sex': 'male'},
 {'pclass': '1st', 'age': 62.0, 'sex': 'male'},
 {'pclass': '3rd', 'age': 31.19418104265403, 'sex': 'female'},
 {'pclass': '1st', 'age': 31.19418104265403, 'sex': 'female'},
 {'pclass': '2nd', 'age': 64.0, 'sex': 'male'},
 {'pclass': '1st', 'age': 31.19418104265403, 'sex': 'female'},
 {'pclass': '3rd', 'age': 24.0, 'sex': 'female'},
 {'pclass': '3rd', 'age': 31.19418104265403, 'sex': 'male'},
 {'pclass': '2nd', 'age': 31.19418104265403, 'sex': 'male'},
 {'pclass': '3rd', 'age': 31.19418104265403, 'sex': 'male'},
 {'pclass': '3rd', 'age': 21.0, 'sex': 'male'},
 {'pclass': '3rd', 'age': 31.19418104265403, 'sex': 'male'},
 {'pclass': '3rd', 'age': 31.19418104265403, 'sex': 'male'},
 {'pclass': '2nd', 'age': 23.0, 'sex': 'female'},
 {'pclass': '3rd', 'age': 31.19418104265403, 'sex': 'male'},
 {'pclass': '3rd', 'age': 31.19418104265403, 'sex': 'female'},
 {'pclass': '3rd', 'age': 31.19418104265403, 'sex': 'female'},
 {'pclass': '1st', 'age': 4

In [121]:
# 进行处理（特征工程）特征 → 类别 → one_hot编码
dict = DictVectorizer(sparse=False)

# 这一步是对字典进行特征抽取,to_dict可以把df变为字典，records代表列名变为键
x_train = dict.fit_transform(x_train.to_dict(orient="records"))
print(type(x_train))
print(dict.get_feature_names_out())
print('-' * 50)
x_test = dict.transform(x_test.to_dict(orient="records"))
print(x_train)

<class 'numpy.ndarray'>
['age' 'pclass=1st' 'pclass=2nd' 'pclass=3rd' 'sex=female' 'sex=male']
--------------------------------------------------
[[30.          0.          1.          0.          0.          1.        ]
 [62.          1.          0.          0.          0.          1.        ]
 [31.19418104  0.          0.          1.          1.          0.        ]
 ...
 [34.          0.          1.          0.          0.          1.        ]
 [46.          1.          0.          0.          0.          1.        ]
 [31.19418104  0.          0.          1.          0.          1.        ]]


In [134]:
# 用决策树进行预测，修改 max_depth 试试,修改 criterion 为 entropy
#树过于复杂，就会产生过拟合
dec = DecisionTreeClassifier()

#训练
dec.fit(x_train, y_train)

# 预测准确率
print("预测的准确率：", dec.score(x_test, y_test))

# 导出决策树的结构
export_graphviz(dec, out_file="tree.dot",
                feature_names=['age', 'pclass=1st', 'pclass=2nd', 'pclass=3rd', 'female', 'male'])


预测的准确率： 0.8085106382978723


In [145]:
#调整决策树的参数
# 分割数据集到训练集合测试集
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=4)
# 进行处理（特征工程）特征-》类别-》one_hot编码
dict = DictVectorizer(sparse=False)

# 这一步是对字典进行特征抽取
x_train = dict.fit_transform(x_train.to_dict(orient="records"))
x_test = dict.transform(x_test.to_dict(orient="records"))

# print(x_train)
# # 用决策树进行预测，修改 max_depth 为10，发现提升了, min_impurity_decrease 带来的增益要大于0.01才会进行划分
dec = DecisionTreeClassifier(max_depth=10,min_impurity_decrease=0.01,min_samples_split=20)

dec.fit(x_train, y_train)
#
# # 预测准确率
print("预测的准确率：", dec.score(x_test, y_test))
#
# # 导出决策树的结构
export_graphviz(dec, out_file="tree1.dot",
                feature_names=dict.get_feature_names_out())

预测的准确率： 0.8206686930091185


In [146]:
y_train.shape

(984,)

In [147]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=4)
# 进行处理（特征工程）特征-》类别-》one_hot编码
dict = DictVectorizer(sparse=False)

# 这一步是对字典进行特征抽取
x_train = dict.fit_transform(x_train.to_dict(orient="records"))
x_test = dict.transform(x_test.to_dict(orient="records"))

In [148]:
# 随机森林进行预测（超参数调优），n_jobs充分利用多核的一个参数
# RandomForestClassifier 是 scikit-learn 中基于 bagging 的集成学习算法，由多棵决策树组成
# n_jobs=-1 表示使用所有可用的 CPU 核心进行并行计算，加速训练
rf = RandomForestClassifier(n_jobs=-1)
# 120, 200, 300, 500, 800, 1200,n_estimators森林中决策树的数目，也就是分类器的数目
# max_samples  是最大样本数
#bagging类型
# n_estimators：森林中决策树的数量（即基分类器个数）。候选值有 1500、2000、5000。
# max_depth：每棵树的最大深度，用于控制树的复杂度，防止过拟合。候选值包括 2、3、5、8、15、25。
param = {"n_estimators": [1500,2000, 5000], "max_depth": [2, 3, 5, 8, 15, 25]}

# 网格搜索与交叉验证
# 将遍历所有超参数组合（共 3×6 = 18 种），对每种组合执行 3 折交叉验证
gc = GridSearchCV(rf, param_grid=param, cv=3)

gc.fit(x_train, y_train)

print("准确率：", gc.score(x_test, y_test))

print("查看选择的参数模型：", gc.best_params_)

print("选择最好的模型是：", gc.best_estimator_)

# print("每个超参数每次交叉验证的结果：", gc.cv_results_)



准确率： 0.8328267477203647
查看选择的参数模型： {'max_depth': 3, 'n_estimators': 1500}
选择最好的模型是： RandomForestClassifier(max_depth=3, n_estimators=1500, n_jobs=-1)
